In [44]:
import spcm
from spcm import units
import numpy as np

# card : spcm.Card

ip = 'TCPIP::192.168.1.83::INSTR'

card = spcm.Card(ip)

card.open(ip)

# setup card for DDS
card.card_mode(spcm.SPC_REP_STD_DDS)

# Setup the channels
channels = spcm.Channels(card)
channels.enable(True)
channels.output_load(50 * units.ohm)
channels.amp(1. * units.V)
card.write_setup()

# trigger mode
trigger = spcm.Trigger(card)
trigger.or_mask(spcm.SPC_TMASK_EXT0) # disable default software trigger
trigger.ext0_mode(spcm.SPC_TM_POS) # positive edge
trigger.ext0_level0(1.5 * units.V) # Trigger level is 1.5 V (1500 mV)
trigger.ext0_coupling(spcm.COUPLING_DC) # set DC coupling
card.write_setup()

# Setup DDS functionality
dds = spcm.DDS(card, channels=channels)
dds.reset()

# dds.data_transfer_mode(spcm.SPCM_DDS_DTM_DMA)

dds.trg_src(spcm.SPCM_DDS_TRG_SRC_CARD)

core_list = [hex(2**n) for n in range(20)]

card.start(spcm.M2CMD_CARD_ENABLETRIGGER)

Exception ignored in: <function Device.__del__ at 0x00000237FFF492D0>
Traceback (most recent call last):
  File "c:\Users\bananas\.venv\kpy\lib\site-packages\spcm\classes_device.py", line 78, in __del__
    self.stop()
  File "c:\Users\bananas\.venv\kpy\lib\site-packages\spcm\classes_device.py", line 321, in stop
    self.cmd(M2CMD_CARD_STOP, *args)
  File "c:\Users\bananas\.venv\kpy\lib\site-packages\spcm\classes_device.py", line 274, in cmd
    self.set_i(SPC_M2CMD, cmd)
  File "c:\Users\bananas\.venv\kpy\lib\site-packages\spcm\classes_device.py", line 440, in set_i
    self._check_error(spcm_dwSetParam_i64(self._handle, register, value))
  File "c:\Users\bananas\.venv\kpy\lib\site-packages\spcm\pyspcm.py", line 337, in spcm_dwSetParam_i64
    return spcm_dwSetParam_i64_ (hDrv, lReg, llVal)
OSError: [WinError -1073741816] Windows Error 0xc0000008
Exception ignored in: <function Device.__del__ at 0x00000237FFF492D0>
Traceback (most recent call last):
  File "c:\Users\bananas\.venv\kpy

In [ ]:
# dds[0].amp(.1)
# dds[0].freq(75.e6)
# dds.exec_at_trg()
# dds.write_to_card()
# dds.data_transfer_mode(spcm.SPCM_DDS_DTM_DMA)


In [45]:
# set up a static tweezer array
frequency_spacing = 1.e6
n_tweezers = 5
f_list = np.arange(75.e6 - n_tweezers*frequency_spacing/2,75.e6 + n_tweezers*frequency_spacing/2, frequency_spacing)
print(f_list)
a_list = [.1,.1,.1,.1,.1]

for tweezer_idx in range(len(core_list)):
    if tweezer_idx < len(f_list):
        dds[tweezer_idx].amp(a_list[tweezer_idx])
        dds[tweezer_idx].freq(f_list[tweezer_idx])
    else:
        dds[tweezer_idx].amp(0.)
dds.exec_at_trg()
dds.write_to_card()
# dds.data_transfer_mode(spcm.SPCM_DDS_DTM_DMA)

[72500000. 73500000. 74500000. 75500000. 76500000.]


In [47]:
# fm the tweezer array for some amount of time, while simultaneously ramping down the modulation depth

# how much time to fm?
fm_time = 5.

# modulation frequency and depth
modulation_freq  = 1.
modulation_depth = 2.e6

# dummy variables to reflect experimental params
initial_optical_power = 5.
final_optical_power = 1.
final_over_initial_optical_power = initial_optical_power / final_optical_power

# rate at which to update awg frequency output ("sample" rate)
period_s = 20.e-3

# number of samples in a period
num_samples = int(1/(period_s * modulation_freq))

# total number of samples
total_samples = fm_time / period_s

# list of sample numbers (in ms)
sample_range = np.arange(total_samples)*period_s

# empty array to be filled with frequency values for each dds core
freq_lists = np.empty([len(f_list),len(sample_range)])

# compute and fill in the above array
for i in range(len(f_list)):
    freq_lists[i] = f_list[i] + modulation_depth * (1 - sample_range*(1 - final_over_initial_optical_power)/(total_samples*period_s))**(1/3) * np.sin(2*np.pi*sample_range*modulation_freq)

# start trigger timer, which outputs trigger events at a given rate
dds.trg_src(spcm.SPCM_DDS_TRG_SRC_TIMER)
dds.trg_timer(period_s)
dds.exec_at_trg()
dds.write_to_card()

# pre_fill the buffer
fill_max = dds.queue_cmd_max()
counter = 0
for counter in range(len(sample_range)):
    for tone_idx in range(np.shape(freq_lists)[0]):
        freq_Hz = freq_lists[tone_idx][counter]
        dds[tone_idx].freq(freq_Hz)
    dds.exec_at_trg()
dds.write_to_card()

# reset trigger mode to external at the end of the fm time
dds.trg_src(spcm.SPCM_DDS_TRG_SRC_CARD)
dds.exec_at_trg()
dds.write_to_card()


In [49]:
# set up another static tweezer array
frequency_spacing = 1.e6
n_tweezers = 3
f_list = np.arange(75.e6 - n_tweezers*frequency_spacing/2,75.e6 + n_tweezers*frequency_spacing/2, frequency_spacing)
print(f_list)
a_list = [.1,.1,.1]

for tweezer_idx in range(len(core_list)):
    if tweezer_idx < len(f_list):
        dds[tweezer_idx].amp(a_list[tweezer_idx])
        dds[tweezer_idx].freq(f_list[tweezer_idx])
    else:
        dds[tweezer_idx].amp(0.)
dds.exec_at_trg()
dds.write_to_card()

[73500000. 74500000. 75500000.]


In [50]:
# use this anytime you want to force a trigger event
trigger.force()

In [51]:
# card.close(card._handle)
card.stop()

card.close(card._handle)